# Tool Agent

In [1]:
!pip install python-dotenv langchain-community wikipedia


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
model_path = os.getenv("MODEL_PATH")

In [3]:
from app.state.final_answer.factory import FinalAnswerNodeFactory
from app.agent.base import BaseAgent
from app.state.defaults import default_tools_handler, default_text_handler
from app.state.node import Node
from app.memory.memory import Memory
from app.persona.persona import Persona
from app.llm.generator import Generator
from app.tools.langchain.wikipedia_query_run import LangchainWikipediaQueryRun
from app.llm.llamacpp.service import LlamaCppService
from typing import Dict, Any, Optional

llm = LlamaCppService(model_path=model_path, n_gpu_layers=-1)
tools = [LangchainWikipediaQueryRun(top_k_results=3, doc_content_chars_max=2048)]


class SearchAgent(BaseAgent):
    name = "SearchAgent"
    description = "Specializes in searching for information."
    pass


final_answer_state = FinalAnswerNodeFactory.get_node(Generator(
    service=llm,
    temperature=0.4,
))

search_results_key = "search_results"


def search_filter_prompt_handler(persona: Persona, memory: Memory, _: Optional[Dict[str, Any]]) -> str:
    messages_formatted = [f"{message.name}: {message.content}\n" for message in memory.data.get_all_messages()]
    search_results = memory.data.pop(search_results_key)
    return f'''{persona.prompt()}

Given the search results, filter out unrelated information that doesn't directly answer the problem. Return a summarized version of the search results with the most important details for the problem.

Current Problem:
"""
{memory.data.get_current_message().content}
"""

Previous Messages:
"""
{messages_formatted}
"""

Search Results:
"""
{search_results}
"""

Give your summarized search results with the most important details for the problem.'''


search_filter_node = Node(
    state_name="search_filter",
    prompt_handler=search_filter_prompt_handler,
    generation_handler=default_text_handler(
        next_state=final_answer_state.state_name,
        exclude_from_scratch_pad=False
    ),
    generator=Generator(
        service=llm,
        use_json_model=True,
        temperature=0.1,
    ),
    tools=tools,
)


def search_prompt_handler(persona: Persona, memory: Memory, tool_schemas: Optional[Dict[str, Any]]) -> str:
    messages_formatted = [f"{message.name}: {message.content}\n" for message in memory.data.get_all_messages()]
    return f'''{persona.prompt()}

Search for information using your tools to help solve the following problem for the user. Use any previous messages as context.

Current Problem:
"""
{memory.data.get_current_message().content}
"""

Previous Messages:
"""
{messages_formatted}
"""

Here are the schemas for the tools you have access to, pick only one:
"""
{tool_schemas}
"""

Respond with the JSON input for the tool of your choice to best solve the problem.'''


search_node = Node(
    state_name="search",
    prompt_handler=search_prompt_handler,
    generation_handler=default_tools_handler(
        next_state=search_filter_node.state_name,
        exclude_from_scratch_pad=True,
        save_data_key=search_results_key,
    ),
    generator=Generator(
        service=llm,
        use_json_model=True,
        temperature=0.1,
    ),
    tools=tools,
)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from D:\llm-models\NousResearch\Hermes-2-Pro-Llama-3-8B-GGUF\Hermes-2-Pro-Llama-3-8B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Hermes-2-Pro-Llama-3-8B
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_m

In [4]:
from app.agent.messages import Query
import logging
import json

logging.basicConfig(level=logging.INFO)

persona = Persona(
    description="You're an autonomous agent who solves problems in natural language. Use your knowledge to best answer the question.")

agent = SearchAgent.start(
    persona=persona,
    memory=Memory(),
    nodes=[
        search_filter_node,
        search_node,
        final_answer_state
    ],
    default_initial_state="search")

future = agent.ask(Query(initial_state="search",
                         goal="Who is the CEO of Microsoft?"))
response = future.get(timeout=30)
print(json.dumps([step.model_dump() for step in response.steps], indent=2))

future = agent.ask(Query(initial_state="search",
                         goal="Where are they from?"))
response = future.get(timeout=30)
print(json.dumps([step.model_dump() for step in response.steps], indent=2))

agent.stop()

INFO:app.agent.base:Agent SearchAgent:urn:uuid:89445e19-e121-4611-ae44-1ce9c43bfc1f received message: Query(goal='Who is the CEO of Microsoft?', initial_state='search', from_caller='user')
INFO:app.agent.base:Agent SearchAgent:urn:uuid:89445e19-e121-4611-ae44-1ce9c43bfc1f executing state: search
from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\<U+0000>-<U+001F>] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a

[
  {
    "name": "search",
    "prompt": "You're an autonomous agent who solves problems in natural language. Use your knowledge to best answer the question.\n\nSearch for information using your tools to help solve the following problem for the user. Use any previous messages as context.\n\nCurrent Problem:\n\"\"\"\nWho is the CEO of Microsoft?\n\"\"\"\n\nPrevious Messages:\n\"\"\"\n['user: Who is the CEO of Microsoft?\\n']\n\"\"\"\n\nHere are the schemas for the tools you have access to, pick only one:\n\"\"\"\n[{'tool_name': 'wikipedia_query_run', 'tool_description': 'A tool for querying the Wikipedia API.', 'tool_parameters': {'description': 'The input to search for factual information on Wikipedia.', 'properties': {'tool_name': {'description': 'The name of the tool you choose to use.', 'title': 'Tool Name', 'type': 'string'}, 'query': {'title': 'Query', 'type': 'string'}}, 'required': ['tool_name', 'query'], 'title': 'WikipediaQueryInput', 'type': 'object'}}]\n\"\"\"\n\nRespond wi


llama_print_timings:        load time =     140.99 ms
llama_print_timings:      sample time =    1448.82 ms /    27 runs   (   53.66 ms per token,    18.64 tokens per second)
llama_print_timings: prompt eval time =     100.33 ms /   266 tokens (    0.38 ms per token,  2651.30 tokens per second)
llama_print_timings:        eval time =     486.80 ms /    26 runs   (   18.72 ms per token,    53.41 tokens per second)
llama_print_timings:       total time =    2217.88 ms /   292 tokens
C:\Users\Alexander\AppData\Local\pypoetry\Cache\virtualenvs\assemble-qevGTaVi-py3.11\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\Alexander\AppD

[
  {
    "name": "search",
    "prompt": "You're an autonomous agent who solves problems in natural language. Use your knowledge to best answer the question.\n\nSearch for information using your tools to help solve the following problem for the user. Use any previous messages as context.\n\nCurrent Problem:\n\"\"\"\nWhere are they from?\n\"\"\"\n\nPrevious Messages:\n\"\"\"\n['user: Who is the CEO of Microsoft?\\n', 'SearchAgent: The current CEO of Microsoft is Satya Narayana Nadella, who took over from Steve Ballmer in 2014. Additionally, he also succeeded John W. Thompson as chairman in 2021.\\n', 'user: Where are they from?\\n']\n\"\"\"\n\nHere are the schemas for the tools you have access to, pick only one:\n\"\"\"\n[{'tool_name': 'wikipedia_query_run', 'tool_description': 'A tool for querying the Wikipedia API.', 'tool_parameters': {'description': 'The input to search for factual information on Wikipedia.', 'properties': {'tool_name': {'description': 'The name of the tool you cho

True

In [5]:
!pip install mermaid-py


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import mermaid as md
from mermaid.graph import Graph

graph: Graph = Graph('example', """
graph TD;
    goal --> search
    search --> search_filter
    search_filter --> final_answer
    final_answer --> exit

    classDef stateNode fill:#fff,stroke:#333,stroke-width:2px,color:#000;
""")
graphe: md.Mermaid = md.Mermaid(graph)
graphe